# Lasso Regression

In [1]:
import numpy as np

In [2]:
def soft_thresholding(x, y):
    return np.sign(x) * max(abs(x) - y, 0)

In [81]:
class Lasso:
    def __init__(self, lambda_, tol=0.0001, max_iter=1000):
        self.lambda_ = lambda_
        self.tor = tol
        self.max_iter = max_iter
        self.w_ = None
        
    def fit(self, X, t):
        n, d = X.shape
        self.w_ = np.zeros(d + 1)
        # instance can be updated by other func. in same class
        avgl1 = 0.
        for _ in range(self.max_iter):
            avgl1_prev = avgl1
            self._update(n, d, X, t)
            # average penalty term
            avgl1 = np.abs(self.w_).sum() / self.w_.shape[0]
            if abs(avgl1 - avgl1_prev) <= self.tor: # little update
                break
            
    def _update(self, n, d, X, t):
        self.w_[0] = (t - np.dot(X, self.w_[1:])).sum() / n
        w0vec = np.ones(n) * self.w_[0]
        for k in range(d):
            ww = self.w_[1:]
            ww[k] = 0
            q = np.dot(t - w0vec - np.dot(X, ww), X[:, k])
            r = np.dot(X[:,k], X[:,k])
            self.w_[k+1] = soft_thresholding(q / r, self.lambda_)
            
            # update self.w_[k+1], not self.w_[k] because 
            # ww[k] corresponds to self.w_[k+1]
            
    def predict(self, X):
        if X.ndim == 1:
            X = X.reshape(X.shape[0], 1)
        Xtil = np.c_[np.ones(X.shape[0]), X]
        return np.dot(Xtil, self.w_)

# Try with win quality dataset

In [82]:
import csv

In [83]:
Xy = []
with open("winequality-red.csv") as fp:
    for row in csv.reader(fp, delimiter=';'):
        Xy.append(row)

Xy = np.array(Xy[1:], dtype=np.float64)

In [84]:
np.random.seed(0)
np.random.shuffle(Xy)
train_X = Xy[:1000, :-1]
train_y = Xy[:1000, -1]
test_x = Xy[1000:, :-1]
test_y = Xy[1000:, -1]

In [87]:
for lambda_ in [1000., 0.1, 0.01]:
    model = Lasso(lambda_)
    model.fit(train_X, train_y)
    y = model.predict(test_x)
    print("---- lambda={} ----".format(lambda_))
    print("coefficiences:")
    print(model.w_)

---- lambda=1000.0 ----
coefficiences:
[ 5.625  0.    -0.     0.     0.    -0.    -0.    -0.    -0.    -0.
  0.     0.   ]
---- lambda=0.1 ----
coefficiences:
[ 5.85311858  0.         -0.33413758  0.48312491  0.         -2.12358859
 -0.         -0.          0.          0.          0.          0.        ]
---- lambda=0.01 ----
coefficiences:
[ 5.77950329  0.         -1.22397907  0.06411323  0.         -3.39086156
 -0.         -0.          0.          0.          1.14739626  0.        ]
